# Title

In [ ]:
# Load packages
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

## Introduction

##### **Background Info:**



##### **Question:**

##### **Dataset Description:**

## Preliminary Exploratory Data Analysis

##### In code cell below:

- Demonstrate that the dataset can be read from the web into R Clean and wrangle your data into a tidy format 
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). 
 - An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). 
 - An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [ ]:
# code cell

## Methods

##### write here

## Expected Outcomes and Significance

##### **What do you expect to find?**

##### **What impact could such findings have?**

##### **What future questions could this lead to?**